In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import skimage.io as io
from skimage import feature
from skimage.transform import resize
from sklearn.feature_selection import VarianceThreshold
from skimage.color import rgb2gray
from sklearn.decomposition import PCA
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from skimage.feature import corner_fast, corner_peaks, corner_harris
from skimage.segmentation import slic
from skimage.color import rgb2lab
from skimage.util import img_as_float
import sys
import os
sys.path.append('../')
import helpers.histogram_classifier as hc

In [2]:
BINS = [i-1 for i in range(1, 250, 8)] + [255]

In [3]:
def image_features(img, segments):
    """
    Find segments in an image and calculate RGB-based features for each segment.
    Specific features calculated are:
    Max, Min, Mean and Range of RGB values
    Normed histograms of RGB values for predfined bins
    :params img: image as numpy array
    :params segments: target number of segments
    :return: numpy array of features of shape (# segments, # features)
    """
    features = []
    segments = slic(img_as_float(img), n_segments=segments, convert2lab=True)
    for s in np.unique(segments):
        f = []
        indices = np.where(segments == s)
        f.append(np.mean(img[indices][:, 0]))
        f.append(np.mean(img[indices][:, 1]))
        f.append(np.mean(img[indices][:, 2]))
        f.append(np.max(img[indices][:, 0]))
        f.append(np.max(img[indices][:, 1]))
        f.append(np.max(img[indices][:, 2]))
        f.append(np.min(img[indices][:, 0]))
        f.append(np.min(img[indices][:, 1]))
        f.append(np.min(img[indices][:, 2]))
        f.append(np.ptp(img[indices][:, 0]))
        f.append(np.ptp(img[indices][:, 1]))
        f.append(np.ptp(img[indices][:, 2]))
        hist_red = np.histogram(img[s][:, 0], bins=BINS, density=True)[0]
        hist_green = np.histogram(img[s][:, 1], bins=BINS, density=True)[0]
        hist_blue = np.histogram(img[s][:, 2], bins=BINS, density=True)[0]
        features.append(np.concatenate([np.array(f), hist_red, hist_green, hist_blue]))
    return np.array(features)

In [4]:
def get_complete_features(image_classes, image_names):
    """
    Get segment-based labels and features for a dictionary of image names
    :params image_classes: list of n image classes
    :params image_names: dictionary of m image names for each class
    :return: features: numpy array of features with one row for each segment of each image
    :return: labels: numpy array of the image class for each segment of of each image
    """
    features = []
    labels = []
    for c in image_classes:
        for i in image_names[c]:
            img = io.imread(os.path.join("../data/food-101/top_classes/", c, "", i))
            image_feats = image_features(img, 100)
            for f in image_feats:
                labels.append(c)
                features.append(f)
    
    return np.array(features), np.array(labels)

In [5]:
from IPython.core.display import HTML
css = open('styles/table-style.css').read() #+ open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [6]:
classes = pd.read_csv('../data/food-101/meta/top_classes.csv', index_col=0)
class_list = list(classes['class'].unique())

image_names = {}
for c in class_list:
    image_names[c] = np.array(classes[classes['class'] == c]['name'])

In [7]:
train, test = hc.split_data(image_names, 0.75)

In [8]:
#for c in class_list:
#    train[c] = train[c][:30]
#    test[c] = test[c][:5]

In [9]:
#Get complete histogram features, edge and corners
train_features, train_labels = get_complete_features(class_list, train)

#### Train Random Forest

In [10]:
#Setup grid-search for top parameters
depth = np.array([8, 10, 12, 14])
estimators = np.array([10, 20, 50, 100, 200])
model = RandomForestClassifier()
grid = GridSearchCV(estimator=model, param_grid=dict(n_estimators=estimators, max_depth=depth))
grid.fit(train_features, train_labels)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': array([ 10,  20,  50, 100, 200]), 'max_depth': array([ 8, 10, 12, 14])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [11]:
print("Best score = {}".format(grid.best_score_))
print("Params: {}".format(grid.best_params_))

Best score = 0.1687375465666438
Params: {'n_estimators': 200, 'max_depth': 14}


In [12]:
%%time
rf = grid.best_estimator_
rf.fit(train_features, train_labels)

CPU times: user 11min 22s, sys: 4.49 s, total: 11min 26s
Wall time: 11min 27s


##### Get test features

In [13]:
test_list = []
for c in class_list:
    for i in test[c]:
        test_image = {}
        img = io.imread(os.path.join("../data/food-101/top_classes/", c, "", i))
        test_image["class"] = c
        test_image["name"] = i
        test_image["features"] = image_features(img, 100)
        test_list.append(test_image)

In [14]:
def predict_image(img_segments, model):
    """
    Predict the class for an image using the segments identified in the image.
    Uses the given classifier to predict the class probabilities for each segment in the image,
    and then returns the class that corresponds to the highest average probability
    
    :params img_segments: numpy array of calculated features for each segment in the image
    :params model: classifier to use
    :return: predicted class name
    """
    predictions = []
    for seg in img_segments:
        predictions.append(model.predict_proba(seg.reshape(1,-1)))
    avg = np.mean(np.array(predictions), axis=0)
    return model.classes_[np.argmax(avg)]

#### Test Random Forest

In [15]:
actual_labels = []
predicted_labels = []
for t in test_list:
    actual_labels.append(t["class"])
    predicted_labels.append(predict_image(t["features"], rf))

In [16]:
overall, metrics, confusion = hc.get_metrics(actual_labels, predicted_labels, class_list)

In [17]:
overall

,Accuracy,F1,Precision,Recall
Results,0.221,0.211,0.23,0.221


In [18]:
metrics

,Precision,Recall,F1-Score,Support
pork_chop,0.31,0.17,0.22,250.0
lasagna,0.27,0.14,0.18,250.0
french_toast,0.20,0.26,0.23,250.0
guacamole,0.21,0.40,0.28,250.0
apple_pie,0.25,0.16,0.20,250.0
cheesecake,0.20,0.37,0.26,250.0
hamburger,0.14,0.15,0.15,250.0
fried_rice,0.35,0.19,0.24,250.0
carrot_cake,0.13,0.05,0.07,250.0
chocolate_cake,0.23,0.29,0.25,250.0


In [19]:
confusion

,pork_chop,lasagna,french_toast,guacamole,apple_pie,cheesecake,hamburger,fried_rice,carrot_cake,chocolate_cake,steak,pizza
pork_chop,43,8,24,26,11,41,10,8,10,23,24,22
lasagna,16,35,33,31,11,30,17,11,9,19,4,34
french_toast,9,11,65,26,14,28,25,8,7,23,10,24
guacamole,5,4,19,101,8,19,23,15,1,17,8,30
apple_pie,8,16,16,27,41,47,31,5,10,21,5,23
cheesecake,3,11,18,21,13,92,16,1,19,41,7,8
hamburger,3,8,28,55,8,29,38,10,6,13,9,43
fried_rice,6,9,22,45,15,19,32,47,9,15,6,25
carrot_cake,11,5,25,31,23,64,18,8,13,28,6,18
chocolate_cake,10,5,19,29,9,43,23,2,9,72,15,14
